<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Prequential Error - Solution

In [1]:
import numpy as np
from sklearn import datasets as skdatasets
from river.stream import iter_sklearn_dataset
from river.naive_bayes import GaussianNB
from river.metrics import Accuracy

In [2]:
iris = skdatasets.load_iris()

## The `progressive_val_score` method

The [progressive_val_score](https://riverml.xyz/latest/api/evaluate/progressive-val-score/) method implements the prequential evaluation. It requires three components: 

- a data stream
- a model
- a metric

In [3]:
from river.evaluate import progressive_val_score

# Setup stream and estimators
stream = iter_sklearn_dataset(iris, shuffle=True, seed=42)
model = GaussianNB()
metric = Accuracy()

# Setup evaluator
progressive_val_score(dataset=stream, model=model, metric=metric, print_every=10)

[10] Accuracy: 44.44%
[20] Accuracy: 73.68%
[30] Accuracy: 82.76%
[40] Accuracy: 84.62%
[50] Accuracy: 85.71%
[60] Accuracy: 88.14%
[70] Accuracy: 88.41%
[80] Accuracy: 89.87%
[90] Accuracy: 89.89%
[100] Accuracy: 90.91%
[110] Accuracy: 91.74%
[120] Accuracy: 92.44%
[130] Accuracy: 92.25%
[140] Accuracy: 92.81%
[150] Accuracy: 91.95%


Accuracy: 91.95%

#### The [progressive_val_score](https://riverml.xyz/latest/api/evaluate/progressive-val-score/) method saves all the predictions in memory and uses all of them to compute the metrics

## Sliding Window Prequential Error
---

The [Rolling](https://riverml.xyz/latest/api/metrics/Rolling/) method is a wrapper that allows you to apply a metric over a window of observations. Under the hood, a buffer with the window_size most recent pairs of (y_true, y_pred) is memorised. When the buffer is full, the oldest pair is removed and the revert method of the metric is called with said pair.

In [4]:
from river.metrics import Rolling

metric = Rolling(Accuracy(), window_size=50)
model = GaussianNB()

for x, y in iter_sklearn_dataset(iris, shuffle=True, seed=42):
    y_p = model.predict_one(x)   # Predict class
    if y_p is not None:
        print(metric.update(y_true=y, y_pred=y_p))
    model.learn_one(x, y)        # Train the model
    

Accuracy: 100.00%	(rolling 50)
Accuracy: 50.00%	(rolling 50)
Accuracy: 33.33%	(rolling 50)
Accuracy: 50.00%	(rolling 50)
Accuracy: 40.00%	(rolling 50)
Accuracy: 50.00%	(rolling 50)
Accuracy: 42.86%	(rolling 50)
Accuracy: 50.00%	(rolling 50)
Accuracy: 44.44%	(rolling 50)
Accuracy: 50.00%	(rolling 50)
Accuracy: 54.55%	(rolling 50)
Accuracy: 58.33%	(rolling 50)
Accuracy: 61.54%	(rolling 50)
Accuracy: 64.29%	(rolling 50)
Accuracy: 66.67%	(rolling 50)
Accuracy: 68.75%	(rolling 50)
Accuracy: 70.59%	(rolling 50)
Accuracy: 72.22%	(rolling 50)
Accuracy: 73.68%	(rolling 50)
Accuracy: 75.00%	(rolling 50)
Accuracy: 76.19%	(rolling 50)
Accuracy: 77.27%	(rolling 50)
Accuracy: 78.26%	(rolling 50)
Accuracy: 79.17%	(rolling 50)
Accuracy: 80.00%	(rolling 50)
Accuracy: 80.77%	(rolling 50)
Accuracy: 81.48%	(rolling 50)
Accuracy: 82.14%	(rolling 50)
Accuracy: 82.76%	(rolling 50)
Accuracy: 83.33%	(rolling 50)
Accuracy: 83.87%	(rolling 50)
Accuracy: 81.25%	(rolling 50)
Accuracy: 81.82%	(rolling 50)
Accuracy:

## Fading Factor Prequential Error
---
It gives more importance to the new predictions and less importance to the oldest ones.
The estimation is updated in this way:

- correct_pred = $\alpha$ * correct_pred + value
- total_seen = $\alpha$ * total_seen + 1.0

with $0 < \alpha \leq 1$. The larger $\alpha$ is, the greater the old predictions importance is.

In [5]:
correct_pred = 0.0
total = 0.0
alpha = 0.95

print_every = 10
i = 1
instance = 1

wrong = 0

model = GaussianNB()

for x, y in iter_sklearn_dataset(iris, shuffle=True, seed=42):
    y_p = model.predict_one(x)   # Predict class
    if y_p is not None:
        to_add = 0.0
        if y == y_p:
            to_add = 1.0                               
        correct_pred = (alpha * correct_pred) + to_add
        total = (alpha * total) + 1.0
   
    if instance == print_every * i:
        if total > 0:
            print('[' + str(instance) + '] Accuracy: ' + str(np.round((correct_pred / total * 100),2)) + '%')
        else:
            print('[' + str(instance) + '] Accuracy: 0.00%')
        i += 1
        
    model.learn_one(x, y)        # Train the model
    instance += 1
    


[10] Accuracy: 43.88%
[20] Accuracy: 80.04%
[30] Accuracy: 90.39%
[40] Accuracy: 90.81%
[50] Accuracy: 91.21%
[60] Accuracy: 94.92%
[70] Accuracy: 91.87%
[80] Accuracy: 95.19%
[90] Accuracy: 92.34%
[100] Accuracy: 95.43%
[110] Accuracy: 97.27%
[120] Accuracy: 98.37%
[130] Accuracy: 94.02%
[140] Accuracy: 96.42%
[150] Accuracy: 88.82%


##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) Quantia Consulting, srl. All rights reserved.